In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
import csv
from sklearn import metrics
from scipy.stats import linregress

#set your own paths
pathdata=r'' #where the predictor and predictand series are stored
pathsave=r'' #where you want the statistics and feature importance information to be stored
cityZ=['den_helder', 'esbjerg', 'gedser', 'Got', 'helsinki', 'KlagshamnMalmoe', 'lowestoft', 'oslo', 'RatanUmeaa']

for icity in cityZ:
    Y = pd.read_csv(pathdata+icity+'_target.csv')
    Y = np.ravel(Y)
    X=pd.read_csv(pathdata+icity+'_predictors.csv')
    RMSE=np.zeros(101)
    corr=np.zeros(101)
    for irand in range(0,101):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=irand)
        model = RandomForestRegressor(bootstrap=True, ccp_alpha=0, max_features = 'sqrt', min_samples_leaf= 2, min_samples_split= 4, n_estimators= 200, max_samples= 0.5)
        model.fit(X_train,Y_train)
        Y_pred=model.predict(X_test) #without feature importance

        #below, for feature importance
        #rfe = RFE(estimator=model, n_features_to_select=35, step=1) #36 = 143 / 4
        #rfe.fit(X_train, Y_train)
        #selected_features = rfe.support_ 
        #model.fit(np.take(X_train, np.ravel(np.where(rfe.support_==True)), axis=1), Y_train)
        #Y_pred=model.predict(np.take(X_test, np.ravel(np.where(rfe.support_==True)), axis=1))
        RMSE[irand]=np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
        _, _, corr[irand], _ , _ = linregress(Y_test, Y_pred)
        #importances = model.feature_importances_
        #features=X.columns.tolist()
        #feat_RFE=np.take(features, np.ravel(np.where(rfe.support_==True)), axis=0)
        #indices = np.argsort(importances)[::-1]
        #outfilename=icity+"_RF_"+str(irand)+".csv"
        #fields = ['Feature','Importance']  
        #with open(str(pathsave)+str(outfilename), 'w') as f:
        #    f.truncate()
        #    writer = csv.writer(f, delimiter='\t')
        #    writer.writerow(fields)
        #    writer.writerows(zip([feat_RFE[i] for i in indices],importances[indices]))  
    fields=['RMSE','corr']
    outfilename=icity+"_RMSE_conf2.csv"
    with open(str(pathsave)+str(outfilename), 'w') as f:
        f.truncate()
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(fields)
        writer.writerows(zip(RMSE,corr))
    print(icity+" RMSE: "+str(np.mean(RMSE))+" "+str(np.min(RMSE))+" CORR: "+str(np.mean(corr))+" "+str(np.max(corr)))